In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Importing the data set
data = pd.read_csv('startups.csv')

In [3]:
# Overview of startup data
print(f'data shape is {data.shape}')
data.head()

data shape is (9349, 75)


,Unnamed: 0,company,n_rounds,startfund,endfund,duration,invest_0,invest_1,invest_2,invest_3,invest_4,invest_5,invest_6,invest_7,invest_8,invest_9,invest_10,invest_11,invest_12,invest_13,invest_14,invest_15,invest_16,invest_17,invest_18,invest_19,invest_20,invest_21,invest_22,invest_23,invest_24,invest_25,invest_26,invest_27,invest_28,invest_29,invest_30,invest_31,time_gap_0-1,time_gap_1-2,time_gap_2-3,time_gap_3-4,time_gap_4-5,time_gap_5-6,time_gap_6-7,time_gap_7-8,time_gap_8-9,time_gap_9-10,time_gap_10-11,time_gap_11-12,time_gap_12-13,time_gap_13-14,time_gap_14-15,time_gap_15-16,time_gap_16-17,time_gap_17-18,time_gap_18-19,time_gap_19-20,time_gap_20-21,time_gap_21-22,time_gap_22-23,time_gap_23-24,time_gap_24-25,time_gap_25-26,time_gap_26-27,time_gap_27-28,time_gap_28-29,time_gap_29-30,time_gap_30-31,Company Status,Accountant,Banking Relationships,Company Founded Date,Total Funding to Date,indicator
0,0,Powertel_Inc,5,1991,1998,7,1.00,1.19,0.59,4.22,0.12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Acquisition,NaN,NaN,05/01/1989,7.12 USD Mil,4.0
1,1,Highpoint_Operating_Corp,9,2002,2008,6,80.00,25.50,33.50,25.00,28.00,30.00,20.00,13.00,20.22,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,1,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Went Public,Deloitte & Touche,NaN,03/01/2002,275.22 USD Mil,1.0
2,2,Spinifex_Pharmaceuticals_Pty_Ltd,13,2005,2015,10,2.48,8.99,5.45,0.15,0.18,3.19,6.86,1.43,0.52,2.60,0.95,45.00,12.51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,1,1,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pending Acquisition,NaN,NaN,06/01/2005,90.31 USD Mil,8.0
3,3,Teleservices_Acquisition_Corp,2,1998,1999,1,22.00,0.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Acquisition,NaN,NaN,NaN,22.45 USD Mil,10.0
4,4,Sotera_Wireless_Inc,13,2005,2017,12,3.00,1.00,0.65,1.04,0.50,20.30,6.70,6.70,10.75,9.71,14.80,20.71,31.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,1,1,1,0,1,2,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Active,NaN,NaN,01/01/2004,127.68 USD Mil,3.0


#### Defining the failure/sucess event as a 0/1 outcome y variable:

In [4]:
data['Company Status'].unique()

array(['Acquisition', 'Went Public', 'Pending Acquisition', 'Active',
       'Defunct', 'Bankruptcy - Chapter 11', 'In Registration', 'Merger',
       'LBO', nan, 'Bankruptcy - Chapter 7', 'Other'], dtype=object)

In [5]:
data['Company Status'].value_counts()

Acquisition                3697
Active                     2158
Defunct                    1694
Went Public                1281
Merger                      180
LBO                         126
Pending Acquisition          59
Bankruptcy - Chapter 11      59
Bankruptcy - Chapter 7       55
In Registration              11
Other                         5
Name: Company Status, dtype: int64

In [6]:
success = {'Acquisition', 'Went Public', 'Merger', 'LBO', 'Pending Acquisition'}
failure = {'Defunct', 'Bankruptcy - Chapter 11','Bankruptcy - Chapter 7'}
def get_status(x):
    if x in success:
        return 1
    elif x in failure:
        return 0
    else:
        return np.nan
data['Company_Status_Label'] = data['Company Status'].apply(get_status)

## drop company labels that are NAN
data.shape
data = data[pd.notna(data['Company_Status_Label'])]
data.shape


(9349, 76)

(7151, 76)

## Applying supervised learning methods:

In [7]:
# Subseting the numerical features
data_numerical = data.select_dtypes(include=['number'])
columns_to_drop = ['startfund', 'endfund']
data_numerical = data_numerical.drop(columns=columns_to_drop)
data_numerical.dtypes;

In [8]:
data_numerical.shape
data_numerical = data_numerical.dropna(how="any")
data_numerical.shape

(7151, 68)

(5266, 68)

#### Train test split, and feature scaling:

In [9]:
from sklearn.model_selection import train_test_split
X = data_numerical.drop(columns=['Company_Status_Label'])
y = data_numerical['Company_Status_Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Logistic regression:

In [10]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print(f'Accuracy of Logistic regression classifier on training set: {logreg.score(X_train, y_train):.2f}')
print(f'Accuracy of Logistic regression classifier on testing set: {logreg.score(X_test, y_test):.2f}')

LogisticRegression()

Accuracy of Logistic regression classifier on training set: 0.80
Accuracy of Logistic regression classifier on testing set: 0.81


#### Decision tree:

In [11]:
from sklearn.tree import DecisionTreeClassifier
dectree = DecisionTreeClassifier()
dectree.fit(X_train, y_train)
print(f'Accuracy of Decision tree classifier on training set: {dectree.score(X_train, y_train):.2f}')
print(f'Accuracy of Decision tree classifier on testing set: {dectree.score(X_test, y_test):.2f}')

DecisionTreeClassifier()

Accuracy of Decision tree classifier on training set: 1.00
Accuracy of Decision tree classifier on testing set: 0.71


#### KNN:

In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print(f'Accuracy of K classifier on training set: {knn.score(X_train, y_train):.2f}')
print(f'Accuracy of KNN classifier on testing set: {knn.score(X_test, y_test):.2f}')

KNeighborsClassifier()

Accuracy of K classifier on training set: 0.83
Accuracy of KNN classifier on testing set: 0.80


/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim